In [1]:
# Code to convert a dataset of comment IDs into comments themselves

# Using PRAW
# API name: sarc_api, ID: Vd1Tea7BOaWAbu99e_s1iQ, secret: st3i2IuHGdfDh7YPY6mVYXlMNJBz1w
# Redirect URI: http://localhost:8080


#Side note
# LLMs being used: GPT (2, 3.5), Claude 3 Sonnet, Gemini Pro, Mixtral, Llama, RoBERTa
# GPT2, Llama, Mixtral, and RoBERTa are accessible through HuggingFace Transformers
# Claude 3 Sonnet is paid (L), Gemini has free versions


In [2]:
import pandas as pd
import praw

import concurrent.futures
#import prawcore
#import time
#import asyncio
#import aiohttp
#import nest_asyncio


reddit = praw.Reddit(client_id='Vd1Tea7BOaWAbu99e_s1iQ', 
                     client_secret='st3i2IuHGdfDh7YPY6mVYXlMNJBz1w', 
                     user_agent='sarc_api',)

In [3]:
data_mtrain_bal = '../data-sarc-llm/main-test-balanced.csv'
df = pd.read_csv(data_mtrain_bal, sep='|', header=None)

df

,0,1,2
0,7x7dx,c07nkao c07nk63,1 0
1,7xtxk,c07pji4 c07ppds,1 0
2,7zh5l,c07twag c07twp2,1 0
3,80nmo,c07y1rj c07xhyn,0 1
4,8139p,c07yhlm c07yoiw,1 0
...,...,...,...
32328,62onuk,dfo7hv8 dfo634e,0 1
32329,62otgi,dfo8kdj dfo8294,1 0
32330,62ovua,dfobpal dfolvl0,1 0
32331,62ozfw,dfowme8 dfoaqdc,0 1


In [4]:
df[['3', '4']] = df[1].str.split(' ', expand=True)

df[['5', '6']] = df[2].str.split(' ', expand=True)

df


,0,1,2,3,4,5,6
0,7x7dx,c07nkao c07nk63,1 0,c07nkao,c07nk63,1,0
1,7xtxk,c07pji4 c07ppds,1 0,c07pji4,c07ppds,1,0
2,7zh5l,c07twag c07twp2,1 0,c07twag,c07twp2,1,0
3,80nmo,c07y1rj c07xhyn,0 1,c07y1rj,c07xhyn,0,1
4,8139p,c07yhlm c07yoiw,1 0,c07yhlm,c07yoiw,1,0
...,...,...,...,...,...,...,...
32328,62onuk,dfo7hv8 dfo634e,0 1,dfo7hv8,dfo634e,0,1
32329,62otgi,dfo8kdj dfo8294,1 0,dfo8kdj,dfo8294,1,0
32330,62ovua,dfobpal dfolvl0,1 0,dfobpal,dfolvl0,1,0
32331,62ozfw,dfowme8 dfoaqdc,0 1,dfowme8,dfoaqdc,0,1


In [7]:
# Concurrently pull data from

new_rows = []
batch = df.iloc[0:8000]

def fetch_comment(reddit, comment_id):
    try:
        comment = reddit.comment(id=comment_id)
        return comment.body
    except Exception as e:
        return None

def process_row(reddit, row):
    body1 = fetch_comment(reddit, row['3'])
    body2 = fetch_comment(reddit, row['4'])
    return [(body1, row['5']), (body2, row['6'])]

with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
        futures = [executor.submit(process_row, reddit, row) for _, row in batch.iterrows()]
        for future in concurrent.futures.as_completed(futures):
            new_rows.extend(future.result())

pd.DataFrame(new_rows, columns=['body', 'sarcastic_label'])


In [ ]:
#from transformers import GPT2Tokenizer


In [6]:
# new_rows = []




# for start in range(0, 60, 20):
#     end = start + 20
#     batch = df.iloc[start:end]


    
#     for i, row in batch.iterrows():
#         cmt = reddit.comment(id=row['3'])
#         new_rows.append([cmt.body, row['5']])

#         cmt = reddit.comment(id=row['4'])
#         new_rows.append([cmt.body, row['6']])
    
# #new_rows = pd.DataFrame(new_rows)
# new_rows